## Goals

Examine how different amounts of predictors affect models and explore different ways to determine the best model to use.

## Import modules

In [11]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm

## Import data

In [2]:
df = pd.read_csv("Craigslist Accommodation.csv")

In [4]:
df.head()

,price,type,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,comes_furnished,laundry_options,parking_options
0,1130,apartment,684,1,1.0,0,0,0,0,0,w/d in unit,carport
1,1190,condo,2190,4,2.0,1,1,1,0,0,NaN,NaN
2,1333,apartment,805,2,1.5,1,1,1,0,0,NaN,NaN
3,2440,apartment,850,1,1.0,0,1,1,0,0,w/d in unit,NaN
4,1250,condo,1500,3,2.0,0,0,1,0,1,w/d in unit,off-street parking


## Fit models

In [5]:
model1 = sm.OLS.from_formula("price ~ type + sqfeet + beds + baths", data = df).fit()

In [6]:
model2 = sm.OLS.from_formula("price ~ type + sqfeet + beds + baths + comes_furnished + laundry_options + parking_options + smoking_allowed", data = df).fit()

In [7]:
model3 = sm.OLS.from_formula("price ~ type + sqfeet + beds + baths + comes_furnished + laundry_options + parking_options + smoking_allowed + cats_allowed + dogs_allowed", data = df).fit()

## Compare r squared

In [8]:
print(model1.rsquared)
print(model2.rsquared)
print(model3.rsquared)

0.12781764439123178
0.28191525276034357
0.2838226722379379


Approximately 28% of variation can be accounted for using model 3, the largest predictor set.

## Compare adjusted r squared

In [10]:
print(model1.rsquared_adj)
print(model2.rsquared_adj)
print(model3.rsquared_adj)

0.12571895013269863
0.2762905680561165
0.27774148003616994


Model 3 still has the highest adusted r squared, suggesting that the extra predictors: cats_allowed and dogs_allowed have a significant effect on the price. Otherwise the adjusted penalty would have eroded their edge.

## Run anova test comparing models

In [12]:
anova_results = anova_lm(model2, model3)
print(anova_results)

   df_resid           ssr  df_diff       ss_diff         F    Pr(>F)
0    3064.0  5.270049e+08      0.0           NaN       NaN       NaN
1    3062.0  5.256050e+08      2.0  1.399862e+06  4.077564  0.017041


With a significance threshold of 0.05, the results of the anova test are significant (Pr>F 0.017). Suggesting that at least one of the predictors: cats_allowed or dogs_allowed is significant and useful to keep in the model.

## Out of sample prediction

#### Find log-likelihood of the models

In [14]:
print(model1.llf)
print(model2.llf)
print(model3.llf)

-37528.12294065279
-22989.87439522474
-22985.76634388421


Model 3 has the highest log-likelihood. This makes sense as it has the most predictors and log-likelihood increases as more predictors are added.

#### Find AIC (Akaike’s information criterion) of the models

In [15]:
print(model1.aic)
print(model2.aic)
print(model3.aic)

75082.24588130559
46029.74879044948
46025.53268776842


Model 3 has the lowest AIC, so in this case we would choose the same model as the previous tests have suggested.

#### Find BIC (Baysean information criterion) of the models

In [17]:
print(model1.bic)
print(model2.bic)
print(model3.bic)

75166.969392794
46180.63885777244
46188.49396047722


Model 2 has the lowest BIC, so in this case we would choose model 2 over model 3 and model 1. This is because BIC values simplicity in the model

## Split housing data

In [21]:
#get the df indices
indices = range(len(df))

#set size variable to 80% of the length of the list of indices
s = int(0.8*len(indices))

#select a random set of indices to train on
train_ind = np.random.choice(indices, size = s, replace = False)

#select the leftover indices to test on
test_ind = list(set(indices) - set(train_ind))

#define the dataframes with their respective indices
df_train = df.iloc[train_ind]
df_test = df.iloc[test_ind]

## Fit models using training data

In [22]:
model2_train = sm.OLS.from_formula("price ~ type + sqfeet + beds + baths + comes_furnished + laundry_options + parking_options + smoking_allowed", data = df_train).fit()

In [23]:
model3_train = sm.OLS.from_formula("price ~ type + sqfeet + beds + baths + comes_furnished + laundry_options + parking_options + smoking_allowed + cats_allowed + dogs_allowed", data = df_train).fit()

## Calculate fitted values

In [24]:
fitted_mod2 = model2_train.predict(df_test)

In [26]:
fitted_mod3 = model3_train.predict(df_test)

## Calculate PRMSE for each model

In [27]:
prmse2 = np.mean((df_test.price - fitted_mod2)**2)**0.5

In [28]:
prmse3 = np.mean((df_test.price - fitted_mod3)**2)**0.5

In [29]:
print(prmse2)
print(prmse3)

444.3451602361372
442.9543681656331


Model 3 has the lowest PRMSE so we'd say that in this case model 3 performs the best.

The process of splitting the data into training and test data involves random choice. Therefore everytime we split the data we will see slightly different prmses.